Mhviraf's [kernel](https://www.kaggle.com/mhviraf/synthetic-data-for-next-instant-gratification) shows the data might be generated by using sklearn's `make_classification`

Using QDA I got a few train rows with high oof score & label 0, or low oof score & label 1.

So I change these data's target to see if it works.

Most part of this notebook is taken from Roman's [kernel](https://www.kaggle.com/nroman/i-m-overfitting-and-i-know-it) & Chris's [kernel](https://www.kaggle.com/cdeotte/pseudo-labeling-qda-0-969)

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from tqdm import tqdm_notebook
import warnings
from sklearn.linear_model import LogisticRegression
import multiprocessing
from scipy.optimize import minimize  
warnings.filterwarnings('ignore')

In [3]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

cols = [c for c in train.columns if c not in ['id', 'target', 'wheezy-copper-turtle-magic']]

In [11]:
oof = np.zeros(len(train))
preds = np.zeros(len(test))

for i in tqdm_notebook(range(512)):

    train2 = train[train['wheezy-copper-turtle-magic'] == i]
    test2 = test[test['wheezy-copper-turtle-magic'] == i]
    idx1 = train2.index
    idx2 = test2.index

    data = pd.concat([pd.DataFrame(train2[cols]), pd.DataFrame(test2[cols])])
    # feature selection
    data2 = VarianceThreshold(threshold = 2).fit_transform(data[cols])

    train3 = pd.DataFrame(data2[:train2.shape[0]], index = idx1)
    train3['target'] = train2['target']
    test3 = pd.DataFrame(data2[train2.shape[0]:], index = idx2)
    
    # Kfold
    skf = StratifiedKFold(n_splits=11, random_state=42)
    for train_index, test_index in skf.split(train3, train3['target']):
        # clf
        #clf = QuadraticDiscriminantAnalysis(0.5)
        X_train = train3.iloc[train_index, :].drop(["target"], axis = 1)
        X_test = train3.iloc[test_index, :].drop(["target"], axis = 1)
        y_train = train3.iloc[train_index, :]['target']
        y_test = train3.iloc[test_index, :]['target']
        #clf.fit(X_train, y_train)
        log = LogisticRegression()
        #clf = QuadraticDiscriminantAnalysis(0.5)
        log.fit(X_train, y_train)
        # output
        train_prob = log.predict_proba(X_train)[:,1]
        test_prob = log.predict_proba(X_test)[:,1]
        oof[idx1[test_index]] = test_prob
        
        # bagging
        preds[idx2] += log.predict_proba(test3)[:,1] / skf.n_splits
        print("Chunk {0} Fold {1}".format(i, roc_auc_score(y_test, test_prob)))

auc = roc_auc_score(train['target'], oof)
print(f'AUC: {auc:.5}')

Chunk 0 Fold 0.9016666666666667
Chunk 0 Fold 0.785
Chunk 0 Fold 0.8016666666666666
Chunk 0 Fold 0.83
Chunk 0 Fold 0.7833333333333333
Chunk 0 Fold 0.8516666666666666
Chunk 0 Fold 0.8099999999999999
Chunk 0 Fold 0.7447916666666667
Chunk 0 Fold 0.8663194444444444
Chunk 0 Fold 0.8541666666666666
Chunk 0 Fold 0.786231884057971
Chunk 1 Fold 0.7518115942028986
Chunk 1 Fold 0.9329710144927535
Chunk 1 Fold 0.9601449275362319
Chunk 1 Fold 0.8840579710144928
Chunk 1 Fold 0.8446969696969697
Chunk 1 Fold 0.8409090909090909
Chunk 1 Fold 0.956439393939394
Chunk 1 Fold 0.884469696969697
Chunk 1 Fold 0.7424242424242424
Chunk 1 Fold 0.9450757575757576
Chunk 1 Fold 0.8712121212121212
Chunk 2 Fold 0.8223062381852552
Chunk 2 Fold 0.7430830039525692
Chunk 2 Fold 0.7628458498023716
Chunk 2 Fold 0.7608695652173914
Chunk 2 Fold 0.7707509881422926
Chunk 2 Fold 0.7114624505928854
Chunk 2 Fold 0.6837944664031621
Chunk 2 Fold 0.9150197628458498
Chunk 2 Fold 0.7292490118577075
Chunk 2 Fold 0.7924901185770751
Chunk 

Chunk 23 Fold 0.7264492753623188
Chunk 23 Fold 0.6231884057971014
Chunk 23 Fold 0.7228260869565217
Chunk 23 Fold 0.661231884057971
Chunk 23 Fold 0.7373188405797101
Chunk 23 Fold 0.8134057971014492
Chunk 23 Fold 0.733695652173913
Chunk 23 Fold 0.6757246376811594
Chunk 24 Fold 0.9280303030303031
Chunk 24 Fold 0.9147727272727273
Chunk 24 Fold 0.8888888888888888
Chunk 24 Fold 0.9761904761904762
Chunk 24 Fold 0.9047619047619047
Chunk 24 Fold 0.8571428571428572
Chunk 24 Fold 0.9404761904761905
Chunk 24 Fold 0.8908730158730158
Chunk 24 Fold 0.873015873015873
Chunk 24 Fold 0.8985507246376812
Chunk 24 Fold 0.7784679089026915
Chunk 25 Fold 0.7784090909090909
Chunk 25 Fold 0.8162878787878788
Chunk 25 Fold 0.7777777777777778
Chunk 25 Fold 0.7738095238095238
Chunk 25 Fold 0.8869047619047619
Chunk 25 Fold 0.8674948240165631
Chunk 25 Fold 0.888198757763975
Chunk 25 Fold 0.8530020703933747
Chunk 25 Fold 0.7826086956521738
Chunk 25 Fold 0.8053830227743272
Chunk 25 Fold 0.8923395445134575
Chunk 26 Fold 

Chunk 47 Fold 0.808695652173913
Chunk 47 Fold 0.7721739130434783
Chunk 47 Fold 0.8226086956521739
Chunk 47 Fold 0.8121739130434784
Chunk 47 Fold 0.8313043478260869
Chunk 47 Fold 0.8125
Chunk 47 Fold 0.8484848484848485
Chunk 47 Fold 0.8806818181818181
Chunk 47 Fold 0.8276515151515152
Chunk 48 Fold 0.75
Chunk 48 Fold 0.735
Chunk 48 Fold 0.8766666666666666
Chunk 48 Fold 0.8783333333333333
Chunk 48 Fold 0.8833333333333333
Chunk 48 Fold 0.7634782608695652
Chunk 48 Fold 0.7060869565217391
Chunk 48 Fold 0.7704347826086957
Chunk 48 Fold 0.7101449275362318
Chunk 48 Fold 0.7844202898550725
Chunk 48 Fold 0.8822463768115942
Chunk 49 Fold 0.8532608695652174
Chunk 49 Fold 0.733695652173913
Chunk 49 Fold 0.7952898550724637
Chunk 49 Fold 0.7300724637681159
Chunk 49 Fold 0.8641304347826088
Chunk 49 Fold 0.8043478260869564
Chunk 49 Fold 0.7228260869565217
Chunk 49 Fold 0.7958412098298676
Chunk 49 Fold 0.8090737240075614
Chunk 49 Fold 0.771266540642722
Chunk 49 Fold 0.7569169960474309
Chunk 50 Fold 0.727

Chunk 71 Fold 0.828125
Chunk 71 Fold 0.8514492753623188
Chunk 71 Fold 0.7463768115942029
Chunk 71 Fold 0.8731884057971014
Chunk 71 Fold 0.7155797101449275
Chunk 71 Fold 0.851449275362319
Chunk 71 Fold 0.9130434782608696
Chunk 71 Fold 0.8623188405797103
Chunk 71 Fold 0.8242753623188406
Chunk 71 Fold 0.8152173913043478
Chunk 72 Fold 0.873015873015873
Chunk 72 Fold 0.8174603174603174
Chunk 72 Fold 0.7658730158730159
Chunk 72 Fold 0.8095238095238095
Chunk 72 Fold 0.7638888888888888
Chunk 72 Fold 0.8154761904761905
Chunk 72 Fold 0.8392857142857143
Chunk 72 Fold 0.873015873015873
Chunk 72 Fold 0.8472222222222222
Chunk 72 Fold 0.7658730158730158
Chunk 72 Fold 0.865424430641822
Chunk 73 Fold 0.7708333333333334
Chunk 73 Fold 0.8072916666666667
Chunk 73 Fold 0.8003472222222222
Chunk 73 Fold 0.8611111111111112
Chunk 73 Fold 0.7447916666666667
Chunk 73 Fold 0.6822916666666666
Chunk 73 Fold 0.8055555555555556
Chunk 73 Fold 0.857638888888889
Chunk 73 Fold 0.7485822306238186
Chunk 73 Fold 0.705103969

Chunk 95 Fold 0.8636363636363635
Chunk 95 Fold 0.7687747035573123
Chunk 96 Fold 0.6918714555765596
Chunk 96 Fold 0.8506616257088847
Chunk 96 Fold 0.8431001890359168
Chunk 96 Fold 0.7977315689981097
Chunk 96 Fold 0.7788279773156899
Chunk 96 Fold 0.7032136105860113
Chunk 96 Fold 0.6994328922495274
Chunk 96 Fold 0.6975425330812854
Chunk 96 Fold 0.6313799621928167
Chunk 96 Fold 0.7845849802371542
Chunk 96 Fold 0.7727272727272727
Chunk 97 Fold 0.7253787878787878
Chunk 97 Fold 0.7291666666666666
Chunk 97 Fold 0.7840909090909092
Chunk 97 Fold 0.8636363636363636
Chunk 97 Fold 0.7430830039525691
Chunk 97 Fold 0.8478260869565217
Chunk 97 Fold 0.9031620553359683
Chunk 97 Fold 0.8467908902691511
Chunk 97 Fold 0.7991718426501035
Chunk 97 Fold 0.8240165631469979
Chunk 97 Fold 0.8074534161490683
Chunk 98 Fold 0.7109090909090909
Chunk 98 Fold 0.8581818181818182
Chunk 98 Fold 0.8345454545454545
Chunk 98 Fold 0.8763636363636365
Chunk 98 Fold 0.7236363636363636
Chunk 98 Fold 0.8290909090909091
Chunk 98 F

Chunk 119 Fold 0.8393194706994328
Chunk 119 Fold 0.8771266540642721
Chunk 119 Fold 0.9073724007561437
Chunk 119 Fold 0.8298676748582231
Chunk 119 Fold 0.9224952741020794
Chunk 119 Fold 0.8063241106719369
Chunk 119 Fold 0.8102766798418972
Chunk 119 Fold 0.7107438016528926
Chunk 119 Fold 0.8243801652892562
Chunk 119 Fold 0.8367768595041322
Chunk 120 Fold 0.8583333333333334
Chunk 120 Fold 0.7166666666666667
Chunk 120 Fold 0.7416666666666667
Chunk 120 Fold 0.86
Chunk 120 Fold 0.63
Chunk 120 Fold 0.8383333333333334
Chunk 120 Fold 0.7634782608695652
Chunk 120 Fold 0.7210144927536232
Chunk 120 Fold 0.7952898550724637
Chunk 120 Fold 0.842391304347826
Chunk 120 Fold 0.822463768115942
Chunk 121 Fold 0.7536231884057971
Chunk 121 Fold 0.7373188405797101
Chunk 121 Fold 0.8731884057971016
Chunk 121 Fold 0.8442028985507246
Chunk 121 Fold 0.8152173913043478
Chunk 121 Fold 0.8242753623188406
Chunk 121 Fold 0.8525519848771267
Chunk 121 Fold 0.8537549407114625
Chunk 121 Fold 0.6600790513833992
Chunk 121 

Chunk 142 Fold 0.9305555555555556
Chunk 142 Fold 0.6440972222222222
Chunk 142 Fold 0.7430555555555556
Chunk 142 Fold 0.736111111111111
Chunk 142 Fold 0.8663194444444444
Chunk 142 Fold 0.7463768115942029
Chunk 142 Fold 0.7590579710144928
Chunk 142 Fold 0.7561436672967864
Chunk 142 Fold 0.7958412098298676
Chunk 143 Fold 0.7047101449275363
Chunk 143 Fold 0.7536231884057971
Chunk 143 Fold 0.6666666666666666
Chunk 143 Fold 0.8659420289855072
Chunk 143 Fold 0.7427536231884058
Chunk 143 Fold 0.8315217391304348
Chunk 143 Fold 0.8106060606060607
Chunk 143 Fold 0.7803030303030303
Chunk 143 Fold 0.8484848484848484
Chunk 143 Fold 0.8295454545454546
Chunk 143 Fold 0.8636363636363635
Chunk 144 Fold 0.808695652173913
Chunk 144 Fold 0.7165217391304347
Chunk 144 Fold 0.6573913043478261
Chunk 144 Fold 0.817391304347826
Chunk 144 Fold 0.9008695652173913
Chunk 144 Fold 0.8191304347826087
Chunk 144 Fold 0.8504347826086956
Chunk 144 Fold 0.7495652173913043
Chunk 144 Fold 0.7391304347826086
Chunk 144 Fold 0.

Chunk 165 Fold 0.722117202268431
Chunk 165 Fold 0.8223062381852553
Chunk 165 Fold 0.9073724007561437
Chunk 165 Fold 0.8166351606805293
Chunk 165 Fold 0.7958412098298676
Chunk 165 Fold 0.7145557655954631
Chunk 166 Fold 0.7218181818181818
Chunk 166 Fold 0.82
Chunk 166 Fold 0.7654545454545455
Chunk 166 Fold 0.5727272727272728
Chunk 166 Fold 0.7836363636363636
Chunk 166 Fold 0.7890909090909091
Chunk 166 Fold 0.8345454545454546
Chunk 166 Fold 0.8552380952380952
Chunk 166 Fold 0.7219047619047619
Chunk 166 Fold 0.6933333333333334
Chunk 166 Fold 0.6646825396825398
Chunk 167 Fold 0.8110367892976589
Chunk 167 Fold 0.8963210702341138
Chunk 167 Fold 0.8765217391304349
Chunk 167 Fold 0.8365217391304348
Chunk 167 Fold 0.8852173913043477
Chunk 167 Fold 0.7704347826086957
Chunk 167 Fold 0.8330434782608696
Chunk 167 Fold 0.7981818181818182
Chunk 167 Fold 0.8436363636363636
Chunk 167 Fold 0.8836363636363637
Chunk 167 Fold 0.8654545454545455
Chunk 168 Fold 0.8043478260869565
Chunk 168 Fold 0.820652173913

KeyboardInterrupt: 

In [16]:
500*512

256000

In [4]:
oof = np.zeros(len(train))
preds = np.zeros(len(test))

for i in tqdm_notebook(range(512)):

    train2 = train[train['wheezy-copper-turtle-magic']==i]
    test2 = test[test['wheezy-copper-turtle-magic']==i]
    idx1 = train2.index; idx2 = test2.index
    train2.reset_index(drop=True,inplace=True)

    data = pd.concat([pd.DataFrame(train2[cols]), pd.DataFrame(test2[cols])])
    data2 = VarianceThreshold(threshold=2).fit_transform(data[cols])

    train3 = data2[:train2.shape[0]]; test3 = data2[train2.shape[0]:]

    skf = StratifiedKFold(n_splits=11, random_state=42)
    for train_index, test_index in skf.split(train2, train2['target']):
        log = LogisticRegression()
        #clf = QuadraticDiscriminantAnalysis(0.5)
        log.fit(train3[train_index,:],train2.loc[train_index]['target'])

        #clf.fit(train3[train_index,:],train2.loc[train_index]['target'])
        
        
        
        oof[idx1[test_index]] = log.predict_proba(train3[test_index,:])[:,1]
        preds[idx2] += log.predict_proba(test3)[:,1] / skf.n_splits

auc = roc_auc_score(train['target'], oof)
print(f'AUC: {auc:.5}')


AUC: 0.80697


In [5]:
train2 = train[train['wheezy-copper-turtle-magic']==i]
test2 = test[test['wheezy-copper-turtle-magic']==i]

In [6]:
test['target'] = preds
test.loc[test['target'] > 0.99, 'target'] = 1
test.loc[test['target'] < 0.01, 'target'] = 0

In [7]:
usefull_test = test[(test['target'] == 1) | (test['target'] == 0)]
new_train = pd.concat([train, usefull_test]).reset_index(drop=True)
new_train.loc[oof > 0.99, 'target'] = 1
new_train.loc[oof < 0.01, 'target'] = 0

In [8]:
oof2 = np.zeros(len(train))
preds2 = np.zeros(len(test))
for i in tqdm_notebook(range(512)):

    train2 = new_train[new_train['wheezy-copper-turtle-magic']==i]
    test2 = test[test['wheezy-copper-turtle-magic']==i]
    idx1 = train[train['wheezy-copper-turtle-magic']==i].index; idx2 = test2.index
    train2.reset_index(drop=True,inplace=True)

    data = pd.concat([pd.DataFrame(train2[cols]), pd.DataFrame(test2[cols])])
    data2 = StandardScaler().fit_transform(VarianceThreshold(threshold=2).fit_transform(data[cols]))
    train3 = data2[:train2.shape[0]]; test3 = data2[train2.shape[0]:]

    skf = StratifiedKFold(n_splits=11, random_state=42)
    for train_index, test_index in skf.split(train2, train2['target']):
        oof_test_index = [t for t in test_index if t < len(idx1)]
        
        log = LogisticRegression()
        #clf = QuadraticDiscriminantAnalysis(0.5)
        log.fit(train3[train_index,:],train2.loc[train_index]['target'])
        if len(oof_test_index) > 0:
            oof2[idx1[oof_test_index]] = log.predict_proba(train3[oof_test_index,:])[:,1]
        preds2[idx2] += log.predict_proba(test3)[:,1] / skf.n_splits
        
auc = roc_auc_score(train['target'], oof2)
print(f'AUC: {auc:.5}')


AUC: 0.80901


In [7]:
sub = pd.read_csv('sample_submission.csv')
sub['target'] = preds2
sub.to_csv('submission.csv',index=False)

It brings ~0.0002 LB boost, be careful using it still.